# Train model using Mouse Brain dataset

In this notebook, we're going to train our model using the Mouse Brain dataset (GSE60361). 

This assumes that you've made the graph using the ```Infer GRN.ipynb``` code.

In [1]:
import os

import numpy as np
import pandas as pd
import torch
import torch_geometric
from torch_geometric.data import Data, Dataset
from tqdm import tqdm
from datasets.datasetMouseBrain import MouseBrainDataset
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from tqdm import tqdm
from scipy.special import softmax
from sklearn.metrics import (auc, precision_recall_curve, roc_auc_score,
                             roc_curve)
from statistics import mean

Torch version: 1.8.0+cu111
Cuda available: True
Torch geometric version: 2.0.3


Load up the dataset. Read ```datasetMouseBrain.py``` on how the dataset was built. 

In [2]:
dataset = MouseBrainDataset("/gpfs/data/rsingh47/hzaki1/data")

100%|██████████| 243075/243075 [00:09<00:00, 26305.11it/s]


In [3]:
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: MouseBrainDataset(3005):
Number of graphs: 3005
Number of features: 1

Data(x=[19972], edge_index=[2, 243075], y=[1])
Number of nodes: 19972
Number of edges: 243075
Average node degree: 12.17
Contains isolated nodes: True
Contains self-loops: False
Is undirected: False


/gpfs/home/hzaki1/celltypefromgrn/env-gpu/lib/python3.7/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'contains_isolated_nodes' is deprecated, use 'has_isolated_nodes' instead
  warnings.warn(out)
/gpfs/home/hzaki1/celltypefromgrn/env-gpu/lib/python3.7/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'contains_self_loops' is deprecated, use 'has_self_loops' instead
  warnings.warn(out)


In [4]:
torch.manual_seed(12345)
#dataset = dataset.shuffle()


shuffle_index = np.loadtxt('shuffle_indices/shuffleIndex_MouseBrain.txt')
shuffle_index = shuffle_index.astype(np.int32)
train_size, val_size = int(len(shuffle_index)* 0.8), int(len(shuffle_index)* 0.9)
train_dataset = [dataset[i] for i in shuffle_index[0:train_size]]
val_dataset = [dataset[i] for i in shuffle_index[train_size: val_size]]
test_dataset =  [dataset[i] for i in shuffle_index[val_size:]]

# train_dataset = torch.load('trainDataset.pt')
# test_dataset = torch.load('testDataset.pt')

# train_dataset = dataset[:2403]
# test_dataset = dataset[2403:]
#test_dataset = dataset[-12:]


# train_dataset = dataset[:40]
# test_dataset = dataset[40:60]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 2404
Number of test graphs: 301


In [5]:
from torch_geometric.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=30, shuffle=True)
train_loader_testing = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

/gpfs/home/hzaki1/celltypefromgrn/env-gpu/lib/python3.7/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [6]:
from gcnmodel import GCN
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GCN(hidden_channels=128, data=dataset, output_size=7).to(device)
print(model)

GCN(
  (conv1): SAGEConv(1, 128)
  (conv2): SAGEConv(128, 128)
  (conv3): SAGEConv(128, 128)
  (lin): Linear(in_features=128, out_features=7, bias=True)
)


In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

    
def test(loader, size):
    model.eval()
    output = np.zeros((len(loader), size))
    actual = np.zeros((len(loader), size))
    accuracy = 0
    for ind, data in enumerate(loader):  # Iterate in batches over the training/test dataset.
        data.x = torch.reshape(data.x, (data.x.shape[0], 1))
        data.x = data.x.type(torch.FloatTensor)
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        output[ind] = softmax(out.cpu().detach().numpy())
        actual[ind][data.y] = 1
        accuracy += int((out.argmax(dim=1) == data.y).sum())
    all_labels = list(dataset.cellToIndex.keys())
    actual = np.array(actual)
    precision = dict()
    recall = dict()
    averageAUROC = []
    averageAUPR = []
    for (idx, c_label) in enumerate(all_labels):
        fpr, tpr, thresholds = roc_curve(actual[:,idx].astype(int), output[:,idx])
        precision[idx], recall[idx], _ = precision_recall_curve(actual[:, idx],
                                                        output[:, idx])
        averageAUROC.append(auc(fpr, tpr))
        averageAUPR.append(round(auc(recall[idx], precision[idx]),4))

    return accuracy/len(loader.dataset), mean(averageAUROC), mean(averageAUPR)


def train():
    model.train()
    avgLoss = 0
    for data in tqdm(train_loader, total=81):  # Iterate in batches over the training dataset.
        data.x = torch.reshape(data.x, (data.x.shape[0], 1))
        data.x = data.x.type(torch.FloatTensor)
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)# Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
        avgLoss += loss
    return avgLoss / 81


for epoch in range(1, 250):
    loss = train()
    train_acc, trainAUC, trainAUPR = test(train_loader_testing, 7)
    test_acc,testAUC, testAUPR = test(val_loader, 7)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Train AUC: {trainAUC:.4f}, Train AUPR: {trainAUPR:.4f}, Test Acc: {test_acc:.4f}, Test Auc: {testAUC:.4f}, Test AUPR: {testAUPR:.4f},  Loss: {loss:.4f}')

100%|██████████| 81/81 [00:09<00:00,  8.44it/s]


Epoch: 001, Train Acc: 0.5017, Train AUC: 0.8089, Train AUPR: 0.3697, Test Acc: 0.4700, Test Auc: 0.7945, Test AUPR: 0.3649,  Loss: 1.6595


100%|██████████| 81/81 [00:09<00:00,  8.70it/s]


Epoch: 002, Train Acc: 0.5391, Train AUC: 0.8563, Train AUPR: 0.4429, Test Acc: 0.5133, Test Auc: 0.8398, Test AUPR: 0.4432,  Loss: 1.3150


100%|██████████| 81/81 [00:09<00:00,  8.67it/s]


...

Epoch: 247, Train Acc: 0.9405, Train AUC: 0.9952, Train AUPR: 0.9462, Test Acc: 0.8300, Test Auc: 0.9700, Test AUPR: 0.7906,  Loss: 0.2813


100%|██████████| 81/81 [00:09<00:00,  8.61it/s]


Epoch: 248, Train Acc: 0.9314, Train AUC: 0.9952, Train AUPR: 0.9475, Test Acc: 0.8267, Test Auc: 0.9701, Test AUPR: 0.7996,  Loss: 0.2781


100%|██████████| 81/81 [00:09<00:00,  8.61it/s]


Epoch: 249, Train Acc: 0.9297, Train AUC: 0.9948, Train AUPR: 0.9420, Test Acc: 0.8233, Test Auc: 0.9701, Test AUPR: 0.7930,  Loss: 0.2954
